
Привет еще раз, меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на красные комментарии ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:


<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>







<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Спасибо за очередную качественную работу - видно, что приложено много усилий,
- Я оставил некоторые советы, надеюсь они будут полезными и интересными
- Есть некоторые недочеты, которые нужно поправить, но у тебя это не должно занять много времени)
- Жду обновленную работу

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление[4]</font>
* Спасибо за усердность!
- Было приятно с тобой сотрудничать.
- Недочеты исправлены - работа полностью корректна
- Не буду больше задерживать, продолжай в том же духе.


# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import timeit
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>

In [2]:
data = pd.read_csv('/datasets/autos.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [3]:
data['time_in_use'] = pd.DatetimeIndex(data['DateCreated']).year - data['RegistrationYear']

Создадим столбец сколько лет авто было на учете до выставления на продажу.

In [4]:
data = data.drop(data[data['time_in_use'] < 0].index)

Удалим строки с машинами поставленными на учет после выставления обьявления

In [5]:
data = data.drop(data[data['RegistrationYear'] < 1980].index)
data = data.drop(data[data['RegistrationYear'] > 2020].index)


<div class="alert alert-block alert-info">
<b>Совет:</b> Советую посмотреть на дату выгрузки анкет - там тоже можно увидеть полезную информацию по поводу границ года регистраци.
</div>

Удалим строки с авто старше 1980

In [6]:
data['Price'].describe()

count    336387.000000
mean       4447.274794
std        4526.680863
min           0.000000
25%        1099.000000
50%        2750.000000
75%        6499.000000
max       20000.000000
Name: Price, dtype: float64

In [7]:
data = data.drop(data[data['Price'] < 200].index)

<div class="alert alert-block alert-success">
<b>Успех:</b>  Самое интересное, что на нескольких сайтов по продажам подержанных автомобилей в Германии (страну можно определить по почтовым индексам, большинство из них из Германии) показало, что цены начинаются действительно с 0 и 1 евро, но это единичные объявления и не понятно, то ли это ошибка при заполнении формы, то ли машины на металлолом. Но я  все-таки склоняюсь к тому, чтобы избавиться от таких данных
</div>

Удалим строки с низкой ценой, иногда такиецены выставляют для попадания в топ.

In [8]:
data['Repaired'] = data['Repaired'].fillna('no_info' )
data['FuelType'] = data['FuelType'].fillna('no_info' )
data['VehicleType'] = data['FuelType'].fillna('no_info' )
data['Gearbox'] = data['Gearbox'].fillna('no_info' )
data['Model'] = data['Model'].fillna('no_info' )
features_no_ohe = data[['RegistrationYear','Power','Kilometer','RegistrationMonth','NumberOfPictures','time_in_use','VehicleType','Gearbox','Model','FuelType','Brand','Repaired']]
target_no_ohe = data['Price']
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320256 entries, 0 to 354368
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        320256 non-null  object
 1   Price              320256 non-null  int64 
 2   VehicleType        320256 non-null  object
 3   RegistrationYear   320256 non-null  int64 
 4   Gearbox            320256 non-null  object
 5   Power              320256 non-null  int64 
 6   Model              320256 non-null  object
 7   Kilometer          320256 non-null  int64 
 8   RegistrationMonth  320256 non-null  int64 
 9   FuelType           320256 non-null  object
 10  Brand              320256 non-null  object
 11  Repaired           320256 non-null  object
 12  DateCreated        320256 non-null  object
 13  NumberOfPictures   320256 non-null  int64 
 14  PostalCode         320256 non-null  int64 
 15  LastSeen           320256 non-null  object
 16  time_in_use        3

Заменим пропуски в данных 'no_info'

<div class="alert alert-block alert-success">
<b>Успех:</b> Пропуски обработаны хорошим образом, а удалить неинформативные признаки - верное решение.
</div>

<div class="alert alert-block alert-info">
<b>Совет:</b> У fuel_type есть категории, которые означают одно и тоже - их можно объединить, либо подумать может они действительно означают, что-то разное.
</div>


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Также есть аномалии в мощности, которые нужно как минимум проанализировать


In [9]:
#data.Power.unique() # 
data = data.drop(data[data['Power'] > 800].index)
data = data.drop(data[data['Power'] < 40].index)
data.Power.unique()

array([190, 163,  75,  69, 102, 109, 125, 101, 105, 140, 115, 131,  60,
       136, 160, 231,  50, 118, 193,  99, 113, 218, 122, 129,  70, 306,
        95,  61, 177, 170,  55, 143, 232, 150, 156,  80,  82,  90, 155,
        54, 185,  87, 180,  86,  84, 224, 235, 200, 178, 265,  77, 110,
       144, 120, 286, 116, 184, 126, 204,  88, 194,  64, 305, 197, 179,
       250,  45, 313,  41, 165,  98, 130, 114, 211,  56, 201, 213,  58,
       107,  83, 174, 100, 220,  73, 192,  68,  66, 299,  74,  52, 147,
       310,  71,  97,  65, 239, 203, 300, 103,  85, 258, 320,  63,  81,
       148, 145, 280, 260, 104, 188, 333, 186, 117, 141, 132, 234, 158,
        92,  51, 135,  59, 230, 209,  43, 146,  67, 106, 166, 276, 344,
        72, 249, 237, 245, 111, 326, 279, 138, 175,  96, 226, 301, 334,
       128, 133, 124, 219, 241, 167,  94, 182, 196, 238, 228, 112,  48,
       272, 139,  79, 256, 243, 252, 396, 340, 121, 345,  44, 187, 289,
       222, 198, 284, 450, 233, 387, 347, 172,  57, 137, 212, 47

<div class="alert alert-block alert-info">
<b>Совет[2]:</b>  Вопрос на подумать: Как думаешь а машины с 0-ой мощностью могут быть просто без двигателя?
</div>



In [10]:
features_ohe = pd.get_dummies(data[['VehicleType','Gearbox','Model','FuelType','Brand','Repaired']], drop_first = True)
data[features_ohe.columns] = features_ohe
features_columns = ['RegistrationYear','Power','Kilometer','RegistrationMonth','NumberOfPictures','time_in_use'] + list(features_ohe.columns)
features_columns

['RegistrationYear',
 'Power',
 'Kilometer',
 'RegistrationMonth',
 'NumberOfPictures',
 'time_in_use',
 'VehicleType_electric',
 'VehicleType_gasoline',
 'VehicleType_hybrid',
 'VehicleType_lpg',
 'VehicleType_no_info',
 'VehicleType_other',
 'VehicleType_petrol',
 'Gearbox_manual',
 'Gearbox_no_info',
 'Model_145',
 'Model_147',
 'Model_156',
 'Model_159',
 'Model_1_reihe',
 'Model_1er',
 'Model_200',
 'Model_2_reihe',
 'Model_300c',
 'Model_3_reihe',
 'Model_3er',
 'Model_4_reihe',
 'Model_500',
 'Model_5_reihe',
 'Model_5er',
 'Model_601',
 'Model_6_reihe',
 'Model_6er',
 'Model_7er',
 'Model_80',
 'Model_850',
 'Model_90',
 'Model_900',
 'Model_9000',
 'Model_911',
 'Model_a1',
 'Model_a2',
 'Model_a3',
 'Model_a4',
 'Model_a5',
 'Model_a6',
 'Model_a8',
 'Model_a_klasse',
 'Model_accord',
 'Model_agila',
 'Model_alhambra',
 'Model_almera',
 'Model_altea',
 'Model_amarok',
 'Model_antara',
 'Model_arosa',
 'Model_astra',
 'Model_auris',
 'Model_avensis',
 'Model_aveo',
 'Model_ayg

<div class="alert alert-block alert-info">
    
<b>Совет:</b> Также как аналог pd.get_dummies хочу посоветовать обратить внимание на OneHotEncoder от sklearn (это более сложный и продвинутый инструмент). Здесь можно посмотреть примеры https://colab.research.google.com/drive/1_gAMXcQKoCShB_l8FNtYEejMnosm9mvt?usp=sharing
</div>






In [11]:
target = data['Price']
features = data[features_columns]


features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)


features_train_no_ohe, features_valid_no_ohe, target_train_no_ohe, target_valid_no_ohe = train_test_split(
    features_no_ohe, target_no_ohe, test_size=0.25, random_state=12345)

Создадим признаки и выборки с данными.

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Нам еще нужна тестовая выборка для независимого тестирования лучшей модели
</div>


Собрали тестовую выборку!

<div class="alert alert-block alert-danger">
    
<b>Ошибка[2]:</b> Нельзя так делать тестовую выборку. Нельзя класть в оба train-test_split одинаковые аргументы `features_no_ohe, target_no_ohe` - иначе получишь песечение выборок
</div>


In [12]:
features_test_no_ohe, features_valid_no_ohe, target_test_no_ohe, target_valid_no_ohe = train_test_split(
    features_valid_no_ohe, target_valid_no_ohe, test_size = 0.25, random_state = 564654)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, test_size=0.25, random_state=12345)

<div class="alert alert-block alert-danger">
    
<b>Ошибка[3]:</b> Выборки для разных кодировок - должны иметь одинаковые размеры. Иначе сравнение получается необъективным, особенно для метрик времени.
</div>


<div class="alert alert-block alert-warning">
<b>Исправил значения test_size  с 0.40 на 0.25</b> 
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[4]:</b> 👍 Только старайся в будущем придерживаться  некоторым общепринятым правилам:
    
- Доля обучающей выборки не меньше 50%
- Доля тестовой и валидационной выборки: от 10% до 25%
- Валидационная и тестовая выборки не отличаются по размеру в два раза или больше раз
</div>

## Обучение моделей

In [67]:
def score(model, target = target_train, features = features_train):
    start_time = timeit.default_timer()
    score = mean_squared_error(target, model.predict(features))**(1/2)
    predict_time = timeit.default_timer() - start_time
    return score, predict_time
def fit(model):
    start_time = timeit.default_timer()
    model.fit(features_train,target_train)
    fit_time = timeit.default_timer() - start_time
    return fit_time

<div class="alert alert-block alert-success">
<b>Успех:</b>  Хорошие помощники
    
</div>

### CatBoostRegressor_no_ohe

In [68]:
parametrs = { 'learning_rate': [0.1*i for i in range(5,11)],
              'iterations': range(60,110,10),}
cats = CatBoostRegressor()
grid = GridSearchCV(cats, parametrs)
grid.fit(features_valid_no_ohe,target_valid_no_ohe, silent= True, cat_features = ['VehicleType','Gearbox','Model','FuelType','Brand','Repaired'])
grid.best_params_

{'iterations': 100, 'learning_rate': 0.5}

In [69]:
cat_no_ohe = CatBoostRegressor(iterations = 100, learning_rate = 0.6000000000000001, depth = 10)
start_time = timeit.default_timer()
cat_no_ohe.fit(features_train_no_ohe,target_train_no_ohe, silent= True, cat_features = ['VehicleType','Gearbox','Model','FuelType','Brand','Repaired'])
cat_no_ohe_fit_time = timeit.default_timer() - start_time
print('время обучения:', cat_no_ohe_fit_time)
Score, time  = score(cat_no_ohe,target_train_no_ohe, features_train_no_ohe)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 19.19602209702134
RMSE score: 1465.8798559020368
время предсказания: 0.4732774421572685


### CatBoostRegressor

In [70]:
cat = CatBoostRegressor(iterations = 500)
start_time = timeit.default_timer()
cat.fit(features_train,target_train, silent= True)
cat_fit_time = timeit.default_timer() - start_time
print('время обучения:', cat_fit_time)
Score, time  = score(cat)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 19.443704338744283
RMSE score: 1550.8488009237758
время предсказания: 0.33264658600091934


### LGBMRegressor

In [71]:
LGBM = LGBMRegressor(iterations = 5, num_leaves = 60, n_estimators=150 )
LGBM_fit_time = fit(LGBM)
print('время обучения:', LGBM_fit_time)
Score, time  = score(LGBM)
print('RMSE score:', Score)
print('время предсказания:', time) 

[LightGBM] [Warning] Unknown parameter: iterations
время обучения: 322.08420442044735
RMSE score: 1543.2406024273616
время предсказания: 4.4506369438022375



<div class="alert alert-block alert-info">
<b>Совет:</b>  У LGBM тоже есть внутренний метод кодировки данных, который можно было бы попробовать
</div>

### DummyRegressor

In [72]:
dummy_regr = DummyRegressor(strategy="median")
dummy_regr_fit_time = fit(dummy_regr)
print('время обучения:', dummy_regr_fit_time)
Score, time  = score(dummy_regr)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 0.003980629146099091
RMSE score: 4908.756744196365
время предсказания: 0.002429317682981491


### LinearRegression

In [73]:
LinearRegr = LinearRegression()
LinearRegr_fit_time = fit(LinearRegr)
print('время обучения:', LinearRegr_fit_time)
Score, time  = score(LinearRegr)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 29.09897469729185
RMSE score: 2479.812031318626
время предсказания: 1.0050266310572624


## Анализ моделей

### LinearRegression

In [74]:
LinearRegr_score, LinearRegr_time  = score(LinearRegr, target = target_valid, features = features_valid)
print('RMSE score:', LinearRegr_score)
print('время предсказания:', LinearRegr_time)

RMSE score: 2487.6588411958887
время предсказания: 0.29796372912824154


### LGBMRegressor

In [75]:
LGBM_score, LGBM_time  = score(LGBM, target = target_valid, features = features_valid)
print('RMSE score:', LGBM_score)
print('время предсказания:', LGBM_time)

RMSE score: 1599.53874069862
время предсказания: 1.5935634169727564


### CatBoostRegressor

In [76]:
cat_score, cat_time  = score(cat, target = target_valid, features = features_valid)
print('RMSE score:', cat_score)
print('время предсказания:', cat_time)

RMSE score: 1595.9654229957284
время предсказания: 0.16142270900309086


### CatBoostRegressor_no_ohe

In [77]:
cat_no_ohe_score, cat_no_ohe_time  = score(cat_no_ohe, target = target_valid_no_ohe, features = features_valid_no_ohe)
print('RMSE score:', cat_no_ohe_score)
print('время предсказания:', cat_no_ohe_time)

RMSE score: 1604.0966630086602
время предсказания: 0.0700276754796505


## Сравнение моделей

In [78]:
columns = ['RMSE_score', 'predict_time', 'fit_time', 'total_time']
index = ['LinearRegression', 'LGBMRegressor', 'CatBoostRegressor', 'CatBoostRegressor_no_ohe']
final_data = [[LinearRegr_score, LinearRegr_time, LinearRegr_fit_time, LinearRegr_time + LinearRegr_fit_time],
              [LGBM_score, LGBM_time, LGBM_fit_time, LGBM_time + LGBM_fit_time],
             [cat_score, cat_time, cat_fit_time, cat_time + cat_fit_time],
              [cat_no_ohe_score, cat_no_ohe_time, cat_no_ohe_fit_time, cat_no_ohe_time + cat_no_ohe_fit_time]]
pd.DataFrame(columns = columns, index = index, data = final_data)

,RMSE_score,predict_time,fit_time,total_time
LinearRegression,2487.658841,0.297964,29.098975,29.396938
LGBMRegressor,1599.538741,1.593563,322.084204,323.677768
CatBoostRegressor,1595.965423,0.161423,19.443704,19.605127
CatBoostRegressor_no_ohe,1604.096663,0.070028,19.196022,19.266050


In [ ]:


cat_score_test, cat_time_test  = score(cat, target = target_test, features = features_test)
print('RMSE score:', cat_score_test)
print('время предсказания:', cat_time_test)


<div class="alert alert-block alert-success">
    
<b>Успех[4]:</b> На тестовой выборке получено хорошее качество!
</div>

# ВЫВОД: 

Мы провели анализ моделей LinearRegression, LGBMRegressor, CatBoostRegressor, CatBoostRegressor_no_ohe.
Вывели в сводную таблицу время обучения, предсказания, качество модели.

Модель CatBoostRegressor  показывает лучший результат по качеству и третий по времени предсказания, обучается довольно быстро. Модель LGBMRegressor предсказывает почти также хорошо, учится по времени дольше всех, но предсказыват по времени хуже всего. 

LinearRegression худшая модель(

<br/>    
<div class="alert alert-info">
<h2> 
    
Лучшая модель градиентного бустинга CatBoostRegressor на тестовой выборке имеет значение метрики оценки качества RMSE = 1591.48	, и время предсказывания 0.09.
<a class="tocSkip"> </h2>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> После анализа и выбора одной наилучшей модели, должно идти  ее тестирование. То есть оценивание на тестовой отложенной выборке
</div>


<br/>    
<div class="alert alert-info">
<h2> 
    
Проверил на тестовой выборке, и подправил выводы!
<a class="tocSkip"> </h2>


<div class="alert alert-block alert-info">
<b>Совет:</b> 

Также если говорить, что можно ещё улучшить в подобных проектах, то я бы выделил такие моменты:<br>
    
1) Для понимания, а какие в итоге факторы важны при моделировании, можно выводить их важность, использую feature_importances_, ну и график заодно. Это будет смотреться очнь здорово: результативная метрика и график важности факторов)))
    
2) У нас разный возраст машин. Есть гипотеза, что для разных возрастов - своё ценообразование. Поэтому, можно попробовать ввести фактор "тип возраста" (ретро, супер-ретро, старая, новая... надо подумать..)..<br>

</div>

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей